In [ ]:
from rmgpy.rmg.main import RMG
from IPython.display import display
import json
import os
from cantherm_input_creation import getExternalSymmetryNumberFromPointGroup

In [ ]:
inputFile = '../polycyclic_qm_inputs/input_total_pg.py'
rmg = RMG()
rmg.loadThermoInput(inputFile)

## Create cantherm input file automatically

In [ ]:
cantherm_input_template = """
#!/usr/bin/env python
# encoding: utf-8

modelChemistry = "M06-2X"
frequencyScaleFactor = 0.99
useHinderedRotors = False
useBondCorrections = False

species('%s', '%s', '%s.py')

statmech('%s')
thermo('%s', 'NASA')
"""

In [ ]:
main_path = os.path.abspath('../polycyclic_cantherm_inputs/')

In [ ]:
# create folder for each species
for spec in rmg.initialSpecies[1:]:
    spec_path = os.path.join(main_path, spec.label)
    if not os.path.exists(spec_path): os.mkdir(spec_path)
    
    # Write input file to disk
    inputFile = open(os.path.join(spec_path,'input.py'),'w')
    inputFile.write(cantherm_input_template % (spec.label, spec.molecule[0].toSMILES(), spec.label, spec.label, spec.label))
    inputFile.close()

## Create cantherm species file automatically

In [ ]:
cantherm_speciesFile_template = """
#!/usr/bin/env python
# -*- coding: utf-8 -*-

atoms = %s

bonds = %s

linear = %s

externalSymmetry = %s

spinMultiplicity = %s

opticalIsomers = %s

energy = {
    'M06-2X': GaussianLog('%s.log'),
}

geometry = GaussianLog('%s.log')

frequencies = GaussianLog('%s.log')

rotors = []
"""

In [ ]:
for spec in rmg.initialSpecies:
    molecule = spec.molecule[0]
    atoms = {}
    for atom in molecule.vertices:
        if atom.symbol not in atoms:
            atoms[atom.symbol] = 1
        else:
            atoms[atom.symbol] += 1
    
    # collect bonds in molecule
    bond_list = []
    for atom1 in molecule.vertices:
        for atom2, bond in molecule.getBonds(atom1).iteritems():
            bond_list.append(bond)
    bond_set = set(bond_list)
    
    # generate bonds dict
    bonds = {}
    for bond in bond_set:
        bond_key = ''
        if bond.order == 'S':
            bond_key = bond.atom1.symbol + '-' + bond.atom2.symbol
        elif bond.order == 'D':
            bond_key = bond.atom1.symbol + '=' + bond.atom2.symbol
        elif bond.order == 'T':
            bond_key = bond.atom1.symbol + '#' + bond.atom2.symbol
        if bond_key == '':
            print 'bond order of {0} cannot be parsed!'.format(bond)
        else:
            if bond_key not in bonds:
                bonds[bond_key] = 1
            else:
                bonds[bond_key] += 1
    
    # get external symmetry number
    extSymNum = getExternalSymmetryNumberFromPointGroup(spec.pointGroup)
    # write into species.py
    spec_path = os.path.join(main_path, spec.label)
    speciesFile = open(os.path.join(spec_path,'{0}.py'.format(spec.label)),'w')
    
    speciesFile.write(cantherm_speciesFile_template % (atoms, bonds, False, extSymNum, 1, 1, spec.label, spec.label, spec.label))
    speciesFile.close()

## Copy log file into right folder

In [ ]:
import shutil

In [ ]:
log_folder = os.path.abspath('../polycyclic_qm_logs/')
for spec in rmg.initialSpecies[1:]:
    log_src = os.path.join(log_folder, '{0}.log'.format(spec.label))
    log_dst = os.path.join(main_path, spec.label, '{0}.log'.format(spec.label))
    if os.path.exists(log_src):
        shutil.copy(log_src, log_dst)
    else:
        print spec.label + ': log file is missing!'

## Run cantherm for each species

In [ ]:
import subprocess
from rmgpy.cantherm.main import CanTherm
from rmgpy.cantherm.thermo import ThermoJob

In [ ]:
import logging
logging.disable(logging.CRITICAL)

cantherm_script_path = "/Users/kehang/Code/rmgmaster/RMG-Py/cantherm.py"
for spec in rmg.initialSpecies:
    spec_path = os.path.join(main_path, spec.label)
    log_path = os.path.join(spec_path, '{0}.log'.format(spec.label))
    if os.path.exists(log_path) and spec.label not in ['s1_5_6_ane','s1_5_6_ene_2', 's1_6_6_ane', 's1_6_6_diene_1_8', 's2_5_6_diene_m_7'] :
        cantherm_input_path = os.path.join(spec_path, 'input.py')
        cantherm = CanTherm()
        cantherm.inputFile = cantherm_input_path
        cantherm.outputDirectory = spec_path
        cantherm.plot = False
        cantherm.execute()
        for job in cantherm.jobList:
            if isinstance(job, ThermoJob):
                spec.thermo = job.species.thermo.toThermoData()
    else:
        print spec.label + ": not running because of no log files!"
    

## Save `ThermoData` of all species into file

In [ ]:
from rmgpy.data.thermo import ThermoLibrary
library = ThermoLibrary(name='Overal Thermo Estimation Library')
for species in rmg.initialSpecies:
    if species.thermo:
        library.loadEntry(
            index = len(library.entries) + 1,
            label = species.label,
            molecule = species.molecule[0].toAdjacencyList(),
            thermo = species.thermo,
            shortDesc = species.thermo.comment,
        )
library.save(os.path.join(main_path,'polycyclic_all_thermoLiabrary.py'))

In [ ]:
## Next steps

- run rmg_wiki/issue/thermo_issues/smart_polycyclic_thermo_estimation.ipynb

- analyze the trend after done the above step

- insert existing data in old polycyclic.py

- run pdd model again

